# Daytona SDK Examples

Load the Daytona SDK and create a sandbox

In [29]:
from daytona_sdk import Daytona
from pprint import pp

daytona = Daytona()

In [ ]:
sandbox = daytona.create()

print(sandbox.id)

## Code and Command Execution

### Code Execution

In [ ]:
response = sandbox.process.code_run('print("Hello World!")')
if response.exit_code != 0:
    print(f"Error: {response.exit_code} {response.result}")
else:
    print(response.result)

### Command Execution

In [ ]:
response = sandbox.process.exec('echo "Hello World from exec!"', cwd="/home/daytona", timeout=10)
if response.exit_code != 0:
    print(f"Error: {response.exit_code} {response.result}")
else:
    print(response.result)

### Exec Sessions

Sessions can be used to execute multiple commands in a single shell that preserves context between commands.

In [ ]:
from daytona_sdk import SessionExecuteRequest

exec_session_id = "exec-session-1"
sandbox.process.create_session(exec_session_id)
session = sandbox.process.get_session(exec_session_id)
pp(session)
print()

# Execute the first command in the session
execCommand1 = sandbox.process.execute_session_command(exec_session_id, SessionExecuteRequest(
    command="export FOO=BAR"
))
if execCommand1.exit_code != 0:
    print(f"Error: {execCommand1.exit_code} {execCommand1.output}")

# Get the command details
session_command = sandbox.process.get_session_command(exec_session_id, execCommand1.cmd_id)
pp(session_command)
print()

# Execute a second command in the session and see that the environment variable is set
execCommand2 = sandbox.process.execute_session_command(exec_session_id, SessionExecuteRequest(
    command="echo $FOO"
))
if execCommand2.exit_code != 0:
    print(f"Error: {execCommand2.exit_code} {execCommand2.output}")
else:
    print(f"Output: {execCommand2.output}\n")

logs = sandbox.process.get_session_command_logs(exec_session_id, execCommand2.cmd_id)
print(f"Logs: {logs}")

## File System

- List Files
- Create Folder
- Upload File
- Download File
- Replace in Files
- Search Files
- Get File Info
- Move Files
- Delete File
- Set File Permissions

In [ ]:
import os
# Get workspace root directory
root_dir = sandbox.get_workspace_root_dir()

# List files in the workspace
files = sandbox.fs.list_files(root_dir)
pp(files)

# Create a new directory in the workspace
new_dir = os.path.join(root_dir, "new-dir")
sandbox.fs.create_folder(new_dir, "755")

file_path = os.path.join(new_dir, "data.txt")

# Add a new file to the workspace
file_content = b"Hello, World!"
sandbox.fs.upload_file(file_path, file_content)

# Search for the file we just added
matches = sandbox.fs.find_files(root_dir, "World!")
pp(matches)

# Replace the contents of the file
sandbox.fs.replace_in_files([file_path], "Hello, World!", "Goodbye, World!")

# Read the file
downloaded_file = sandbox.fs.download_file(file_path)
print("File content:", downloaded_file.decode("utf-8"))

# Change the file permissions
sandbox.fs.set_file_permissions(file_path, mode="777")

# Get file info
file_info = sandbox.fs.get_file_info(file_path)
pp(file_info)  # Should show the new permissions

# Move the file to the new location
new_file_path = os.path.join(root_dir, "moved-data.txt")
sandbox.fs.move_files(file_path, new_file_path)

# Find the file in the new location
search_results = sandbox.fs.search_files(root_dir, "moved-data.txt")
pp(search_results)

# Delete the file
sandbox.fs.delete_file(new_file_path)

## Git

- Clone Repository
- Pull Repository
- List Branches
- Delete a Branch
- Create a Branch
- Checkout a Branch
- Git Log
- Git Status
- Git Add
- Git Commit

In [ ]:
root_dir = sandbox.get_workspace_root_dir()
project_dir = os.path.join(root_dir, "learn-typescript")

# Clone the repository
sandbox.git.clone(
    "https://github.com/panaverse/learn-typescript", project_dir, "master"
)

sandbox.git.pull(project_dir)

branches = sandbox.git.branches(project_dir)
pp(branches)

## LSP

- Start Language Server
- Notify Language Server of Document Change
- Get Completions
- Document Symbols
- Workspace Symbols

In [ ]:
root_dir = sandbox.get_workspace_root_dir()
project_dir = os.path.join(root_dir, "learn-typescript")

# Search for the file we want to work on
matches = sandbox.fs.find_files(project_dir, "var obj1 = new Base();")
print("Matches:", matches)

# Start the language server
lsp = sandbox.create_lsp_server("typescript", project_dir)
lsp.start()

# Notify the language server of the document we want to work on
lsp.did_open(matches[0].file)

# Get symbols in the document
symbols = lsp.document_symbols(matches[0].file)
print("Symbols:", symbols)

# Fix the error in the document
sandbox.fs.replace_in_files(
    [matches[0].file], "var obj1 = new Base();", "var obj1 = new E();"
)

# Notify the language server of the document change
lsp.did_close(matches[0].file)
lsp.did_open(matches[0].file)

# Get completions at a specific position
completions = lsp.completions(matches[0].file, {"line": 12, "character": 18})
print("Completions:", completions)

## Sandbox Management

- List Sandboxes
- Stop Sandbox
- Start Sandbox
- Remove Sandbox

In [ ]:
sandboxes = daytona.list()
print(f"Total sandboxes count: {len(sandboxes)}")

for s in sandboxes:
    print(s.id)
    pp(s.info())
    print()

In [38]:
daytona.stop(sandbox)

In [39]:
daytona.start(sandbox)

In [40]:
daytona.remove(sandbox)